In [8]:
pip install nltk

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
pip install nltk

In [11]:
import nltk

In [12]:
import pandas as pd

In [13]:
#Upload the csv files
fake = pd.read_csv('Fake.csv')
gen = pd.read_csv('True.csv')

In [14]:
fake['target']=0
gen['target']=1

In [15]:
#concate both the files and reset the index
data = pd.concat([fake,gen],axis=0)
data = data.reset_index(drop=True)
data.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [16]:
data = data.drop(['subject','date','title'],axis=1)

In [17]:
print(data.columns)

Index(['text', 'target'], dtype='object')


In [22]:
#Tokenization
from nltk.tokenize import word_tokenize
data['text']=data['text'].apply(word_tokenize)

In [23]:
print(data.head())

                                                text  target
0  [Donald, Trump, just, couldn, t, wish, all, Am...       0
1  [House, Intelligence, Committee, Chairman, Dev...       0
2  [On, Friday, ,, it, was, revealed, that, forme...       0
3  [On, Christmas, day, ,, Donald, Trump, announc...       0
4  [Pope, Francis, used, his, annual, Christmas, ...       0


In [24]:
#Stemmming
from nltk.stem.snowball import SnowballStemmer

In [25]:
porter =SnowballStemmer('english')

In [26]:
def stemit(text):
  return[porter.stem(word) for word in text]

In [27]:
data['text']=data['text'].apply(stemit)
print(data.head())

                                                text  target
0  [donald, trump, just, couldn, t, wish, all, am...       0
1  [hous, intellig, committe, chairman, devin, nu...       0
2  [on, friday, ,, it, was, reveal, that, former,...       0
3  [on, christma, day, ,, donald, trump, announc,...       0
4  [pope, franci, use, his, annual, christma, day...       0


In [28]:
#Stopword Removal
def stopit(t):
  dt=[word for word in t if len(word)>2]
  return dt

In [29]:
data['text']=data['text'].apply(stopit)

In [30]:
print(data.head())

                                                text  target
0  [donald, trump, just, couldn, wish, all, ameri...       0
1  [hous, intellig, committe, chairman, devin, nu...       0
2  [friday, was, reveal, that, former, milwauke, ...       0
3  [christma, day, donald, trump, announc, that, ...       0
4  [pope, franci, use, his, annual, christma, day...       0


In [31]:
data['text']=data['text'].apply(' '.join)

In [32]:
data.head()

,text,target
0,donald trump just couldn wish all american hap...,0
1,hous intellig committe chairman devin nune hav...,0
2,friday was reveal that former milwauke sheriff...,0
3,christma day donald trump announc that would b...,0
4,pope franci use his annual christma day messag...,0


In [33]:
#Splitting
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['text'],data['target'])
display(X_train.head())
print('\n')
display(y_train.head())

42361    kabul reuter nato countri have strong reason m...
37803    tokyo reuter japanes prime minist shinzo abe t...
13819    girl aren the onli gender who will suffer emba...
18953    former senat jim demint speak the wsj about th...
30181    new york/washington reuter president-elect don...
Name: text, dtype: object

42361    1
37803    1
13819    0
18953    0
30181    1
Name: target, dtype: int64

In [34]:
#Vetorisation
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer(max_df = 0.7)
tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.fit_transform(X_test) 

In [38]:
print(tfidf_test)

  (0, 20659)	0.04759573973957933
  (0, 38169)	0.07313759593966443
  (0, 20740)	0.09932428183470883
  (0, 54018)	0.10784368922654483
  (0, 51478)	0.03275622372617735
  (0, 24378)	0.06933220131164083
  (0, 18591)	0.03649166401629601
  (0, 48384)	0.039532702653895066
  (0, 9353)	0.07102074187567171
  (0, 50533)	0.058874538399581906
  (0, 45316)	0.08312555361639651
  (0, 43438)	0.04505762261797623
  (0, 10973)	0.0717175993554705
  (0, 392)	0.09257789674459205
  (0, 36184)	0.026951574577364423
  (0, 5592)	0.04927444061236931
  (0, 40210)	0.09042000037903443
  (0, 53105)	0.04525722376306563
  (0, 50395)	0.06465120762825866
  (0, 25572)	0.035798124576323184
  (0, 48817)	0.041089303621827666
  (0, 6045)	0.058501481815388404
  (0, 36738)	0.052898040373081746
  (0, 28927)	0.08080424971689056
  (0, 51364)	0.11148937793524892
  :	:
  (11224, 5481)	0.03288769621775411
  (11224, 15109)	0.03952706660047508
  (11224, 36073)	0.015387841949477878
  (11224, 42851)	0.030758615096957178
  (11224, 52625)	0.

In [39]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train,y_train)
pred_1 = model_1.predict(tfidf_test)
cr1=accuracy_score(y_test,pred_1)
print(cr1*100)

In [41]:
#Passive-Aggresive Classifier
from sklearn.linear_model import PassiveAggressiveClassifier
model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train,y_train)


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test,y_pred)
print("The accuracy of the Prediction is ",accscore*100)